# Instructions

The assignment consists of three tasks:

- Run the T-test for the means of two independent samples underlying the statement "IRE binding activity was significantly reduced in failing hearts" (originally published by Haddad et al. in https://doi.org/10.1093/eurheartj/ehw333) using the following example data.

| non-failing heart (NF) | failing heart (F) |
| ---------------------- | ----------------- |
| 95 | 50 |
| 103 | 35 |
| 99 | 21 | 
| &nbsp; | 15 | 
| &nbsp; | 7 | 
| &nbsp; | 40 |

- Describe the statistical hypothesis test in machine readable form following the [statistical methods ontology concept for "two sample t-test with unequal variance"](https://www.ebi.ac.uk/ols/ontologies/stato/terms?iri=http%3A%2F%2Fpurl.obolibrary.org%2Fobo%2FSTATO_0000304) using Semantic Web technologies, namely the Resource Description Framework (RDF)
- Process the resulting machine readable description using Semantic Web technologies, namely the SPARQL Protocol and RDF Query Language.

Please return the assignment with all outputs visible (i.e., do not clear the outputs).

Good luck!

In [2]:
#!pip install rdflib pandas scipy numpy

In [1]:
# Import all required libraries (some are missing)
import pandas as pd
from scipy import stats
from rdflib import Graph, URIRef
from rdflib.namespace import RDF,RDFS,FOAF
from rdflib import BNode, Literal
from pprint import pformat

In [2]:
# Run the T-test for the means of two independent samples using the example data

# load the csv file containing the example data
file=pd.read_csv(r"C:\Users\Arslan Amjad\Desktop\ttest.csv")
data=file.head(10)

# labels seperation
labels=data.head(0)

# NF -- data items
NF_data=data.loc[:,'NF']
NF_data=NF_data.dropna()

# F -- data items
F_data=data.loc[:,'F']

# T-test based on two independent samples
results=stats.ttest_ind(NF_data,F_data)
# print the stats and p-value
print(results)

Ttest_indResult(statistic=7.182560914224794, pvalue=0.00018016897790311737)


In [3]:
# Describe the statistical hypothesis test in machine readable form

# First, we initialize an RDF Graph and bind some prefixes
g = Graph()
g.bind('obo', 'http://purl.obolibrary.org/obo/')
g.bind('ex', 'http://example.org/')

# Next, we define some needed vocabulary
obo = dict()
obo['two sample t-test with unequal variance'] = URIRef('http://purl.obolibrary.org/obo/STATO_0000304')
obo['has specified input'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000293')
obo['has specified output'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000299')
obo['p-value'] = URIRef('http://purl.obolibrary.org/obo/OBI_0000175')
obo['has value specification'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001938')
obo['scalar value specification'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001931')
obo['has specified numeric value'] = URIRef('http://purl.obolibrary.org/obo/OBI_0001937')
obo['iron-responsive element binding'] = URIRef('http://purl.obolibrary.org/obo/GO_0030350')

# Now, populate the graph with statements
# As an example, the following statement types the resource (blank node) 'n1' as 'two sample t-test with unequal variance'
n1 = BNode()
g.add((n1, RDF.type, obo['two sample t-test with unequal variance']))

# Now, state that the t-test has iron-responsive element binding as a specified input (one statement to complete)
n2 = BNode()
g.add((n1, obo['has specified input'], n2))
n3=BNode(results[0])
g.add((n1,obo['has specified input'],n3))

# Next, state that the t-test has the p-value computed above as a specified output (two statements to complete)
n4=BNode()
g.add((n4,obo['has specified output'],n3))
n5=BNode()
g.add((n5,obo['has specified output'],n4))
print(g.all_nodes())
# The literal value of the p-value is represented as a scalar value specification with a specified numeric value (three statements to complete)
n=URIRef('http://purl.obolibrary.org/obo/OBI_0000175')
n6=BNode()
name=Literal('p-value')
value=Literal(results[1])
RDF.type
FOAF.knows
pvalue=Literal((n,RDF.type,n6))


g.add((n,RDF.type,name))
g.add((n,RDF.type,value))
# Type the literal as double
#value=Literal(results[1],datatype=XSD.double)
#g.add((n, obo['has specified numeric value'], value)

# Finally, serialize and print the graph in RDF/XML format
print(g.serialize(format='turtle'))
print('\n')

{rdflib.term.URIRef('http://purl.obolibrary.org/obo/STATO_0000304'), rdflib.term.BNode('N2ebf0edf418a4e69900544edbd99ff62'), rdflib.term.BNode('N7748689d2cec428195c03cc8935cd547'), rdflib.term.BNode('N5d87caff3f87432182a95d782f60b5ec'), rdflib.term.BNode('7.182560914224794'), rdflib.term.BNode('Nbb0ce5ea1cf141278e0214af66eec63a')}
b'@prefix ex: <http://example.org/> .\n@prefix obo: <http://purl.obolibrary.org/obo/> .\n@prefix rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#> .\n@prefix rdfs: <http://www.w3.org/2000/01/rdf-schema#> .\n@prefix xml: <http://www.w3.org/XML/1998/namespace> .\n@prefix xsd: <http://www.w3.org/2001/XMLSchema#> .\n\nobo:OBI_0000175 a 1.80169e-04,\n        "p-value" .\n\n[] obo:OBI_0000299 [ obo:OBI_0000299 _:7.182560914224794 ] .\n\n[] a obo:STATO_0000304 ;\n    obo:OBI_0000293 _:7.182560914224794,\n        [ ] .\n\n'




In [4]:
# Process the machine readable statistical hypothesis test by completing the following SPARQL query that returns the p-value
q=g.query("""
#PREFIX obo: <http://purl.obolibrary.org/obo/>
  SELECT ?reults[1] 
  WHERE {
    ?r a obo:STATO_0000304 .
    ?r obo:OBI_0000293 [ a obo:GO_0030350 ] .
    ?r obo:OBI_0000299 [
    # To be completed ...
    ]
  }""")

print("/n ========Machine readable hypothesis==========")
for qs in g.query(q):
    print(qs[0])

ParseException: Expected {SelectQuery | ConstructQuery | DescribeQuery | AskQuery} (at char 64), (line:3, col:17)